In [14]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from urllib.request import urlopen
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [ ]:
import mysql.connector
remote = mysql.connector.connect(
    host = "****",
    port = 3306,
    user = "root",
    password = "****",
    database = "Encar"
)

In [9]:
cur = remote.cursor()

In [11]:
class Info:
    def __init__(self):
        self.brand_name = None
        self.model_name = None
        self.price = None
        self.accident = None
        self.year = None
        self.distance = None
        self.cc = None
        self.kind = None
        self.transmission = None
        self.fueltype = None

    def sql_query(self):
        sql = "insert into Info(brand, name, price, accident, year, distance, cc, kind, transmission, fueltype) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        return sql
    
    def __del__(self):
        print("=*= Car Deleted =*=")

In [ ]:
url = "http://www.encar.com/fc/fc_carsearchlist.do?carType=for#!"
driver.get(url)

driver.maximize_window()
pages_list = driver.find_element(By.ID, "pagination")
pages = pages_list.find_elements(By.TAG_NAME, "span")
pages = pages[:-1]

for i in range(len(pages)):
    pages_list = driver.find_element(By.ID, "pagination")
    pages = pages_list.find_elements(By.TAG_NAME, "span")
    print(pages[i].text)

    pages[i].click()
    time.sleep(1)

    models = driver.find_elements(By.CLASS_NAME, "car_list")
    for model in models:
        print("===================")
        try:
            infos = model.find_elements(By.CLASS_NAME, "cls")
            for info in infos:
                car = Info()
                brand_name = info.find_element(By.TAG_NAME, "strong")
                model_name = info.find_element(By.TAG_NAME, "em")
                car.brand_name = brand_name.text
                car.model_name = model_name.text

                print(brand_name.text)
                print(model_name.text)
                
                driver.execute_script("arguments[0].scrollIntoView();", info)
                info.click()

                time.sleep(0.5)

                """new tab opened"""
                original_window = driver.current_window_handle
                wait = WebDriverWait(driver, 2)
                wait.until(lambda d: len(d.window_handles) > 1)

                new_window = [window for window in driver.window_handles if window != original_window][0]
                driver.switch_to.window(new_window)

                try:
                    Detail = driver.find_element(By.CLASS_NAME, "DetailSummary_btn_detail__msm-h")
                    price = driver.find_element(By.CLASS_NAME, "DetailLeadCase_price__tfeps")
                    accident = driver.find_element(By.CLASS_NAME, "EncarDiagnosisPanel_link__9ZR72")
                    price = price.text.replace("만원", '')
                    price = int(price.replace(',', ''))*10000
                    car.price = price
                    car.accident = accident.text

                    print(price)
                    print(accident.text)
                    Detail.click()

                    try:
                        info_box = driver.find_element(By.ID, "bottom_sheet")
                        infos = info_box.find_element(By.TAG_NAME, "ul")
                        info = infos.find_elements(By.TAG_NAME, "li")
                        cnt = 0
                        for each in info:
                            cnt += 1
                            match cnt:
                                case 2:
                                    needs = each.find_element(By.CLASS_NAME, "DetailSpec_txt__NGapF")
                                    year = needs.text.split(" ")[0]
                                    year = int(year.replace('년', ''))
                                    car.year = year
                                    print(car.year)
                                case 3:
                                    needs = each.find_element(By.CLASS_NAME, "DetailSpec_txt__NGapF")
                                    distance = needs.text.replace("km", '')
                                    try:
                                        distance = int(distance.replace(",", ''))
                                    except:
                                        pass
                                    car.distance = distance
                                    print(car.distance)
                                case 4:
                                    needs = each.find_element(By.CLASS_NAME, "DetailSpec_txt__NGapF")
                                    cc = needs.text.replace(",", '')
                                    cc = int(cc.replace("cc", ''))
                                    car.cc = cc
                                    print(car.cc)
                                case 5:
                                    needs = each.find_element(By.CLASS_NAME, "DetailSpec_txt__NGapF")
                                    car.fueltype = needs.text
                                    print(car.fueltype)
                                case 6:
                                    needs = each.find_element(By.CLASS_NAME, "DetailSpec_txt__NGapF")
                                    car.transmission = needs.text
                                    print(car.transmission)
                                case 7:
                                    needs = each.find_element(By.CLASS_NAME, "DetailSpec_txt__NGapF")
                                    car.kind = needs.text
                                    print(car.kind)

                        try:
                            sql_text = car.sql_query()
                            cur.execute(sql_text, (car.brand_name, car.model_name, car.price, car.accident, car.year, car.distance, car.cc, car.kind, car.transmission, car.fueltype))
                            remote.commit()
                        except:
                            print("SQL Error")
                    except:
                        print("Detailed Info Error")

                except:
                    print("Price or Accident Error")
                
                del car # delete instance
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
        except:
            print("Error")

In [16]:
remote.close()

---

In [2]:
import pandas as pd

query = "SELECT * FROM CarInfo;"
df = pd.read_sql(query, remote)
remote.close()

df

/tmp/ipykernel_88420/2482947094.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, remote)


,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype
0,현대,더 뉴 그랜저 IG,26900000,프레임무사고 확인,20,26143,2497,대형차,오토,가솔린
1,기아,K5 3세대,27900000,프레임무사고 확인,22,20222,1598,중형차,오토,가솔린
2,기아,더 뉴 레이,12600000,프레임무사고 확인,20,15939,998,경차,오토,가솔린
3,제네시스,G80 (RG3),37500000,프레임무사고 확인,21,84011,2497,대형차,오토,가솔린
4,쉐보레(GM대우),더 넥스트 스파크,5400000,프레임무사고 확인,16,107792,999,경차,오토,가솔린
...,...,...,...,...,...,...,...,...,...,...
477,벤츠,E-클래스 W213,46990000,프레임무사고 확인,21,31579,1950,중형차,오토,디젤
478,BMW,X1 (E84),8200000,프레임무사고 확인,11,99995,1995,SUV,오토,디젤
479,미니,쿠퍼,12300000,프레임무사고 확인,15,54594,1499,소형차,오토,가솔린
480,폭스바겐,골프 7세대,8790000,프레임무사고 확인,15,142141,1395,준중형차,오토,가솔린


예약중 가격인 '9,999만원'은 df에서 삭제

In [3]:
df[df['price']==99990000]

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype
37,현대,더 뉴 코나,99990000,프레임무사고 확인,21,27598,1598,SUV,오토,가솔린
185,현대,더 뉴 아반떼 (CN7),99990000,프레임무사고 확인,23,10330,1598,준중형차,오토,가솔린
192,기아,K7 프리미어,99990000,프레임무사고 확인,21,56956,2497,대형차,오토,가솔린
228,벤츠,GLC-클래스 X253,99990000,프레임무사고 확인,22,37449,1950,SUV,오토,디젤
383,BMW,5시리즈 (G30),99990000,프레임무사고 확인,22,47292,1998,중형차,오토,가솔린
463,재규어,F-PACE,99990000,프레임무사고 확인,17,176182,1999,SUV,오토,디젤


In [4]:
df = df.drop(df[df['price']==99990000].index)

df

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype
0,현대,더 뉴 그랜저 IG,26900000,프레임무사고 확인,20,26143,2497,대형차,오토,가솔린
1,기아,K5 3세대,27900000,프레임무사고 확인,22,20222,1598,중형차,오토,가솔린
2,기아,더 뉴 레이,12600000,프레임무사고 확인,20,15939,998,경차,오토,가솔린
3,제네시스,G80 (RG3),37500000,프레임무사고 확인,21,84011,2497,대형차,오토,가솔린
4,쉐보레(GM대우),더 넥스트 스파크,5400000,프레임무사고 확인,16,107792,999,경차,오토,가솔린
...,...,...,...,...,...,...,...,...,...,...
477,벤츠,E-클래스 W213,46990000,프레임무사고 확인,21,31579,1950,중형차,오토,디젤
478,BMW,X1 (E84),8200000,프레임무사고 확인,11,99995,1995,SUV,오토,디젤
479,미니,쿠퍼,12300000,프레임무사고 확인,15,54594,1499,소형차,오토,가솔린
480,폭스바겐,골프 7세대,8790000,프레임무사고 확인,15,142141,1395,준중형차,오토,가솔린


정상 삭제 확인

In [5]:
df[df['price']==99990000]

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype


------

사고 여부가 의심스럽다. 모든 매물이 무사고일 것 같다.

In [6]:
df[df['accident']!="프레임무사고 확인"]

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype


유사고 차량이 없다. 사고 여부는 예측 성능에 도움을 주지 않을 것으로 예상된다.

In [7]:
df

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype
0,현대,더 뉴 그랜저 IG,26900000,프레임무사고 확인,20,26143,2497,대형차,오토,가솔린
1,기아,K5 3세대,27900000,프레임무사고 확인,22,20222,1598,중형차,오토,가솔린
2,기아,더 뉴 레이,12600000,프레임무사고 확인,20,15939,998,경차,오토,가솔린
3,제네시스,G80 (RG3),37500000,프레임무사고 확인,21,84011,2497,대형차,오토,가솔린
4,쉐보레(GM대우),더 넥스트 스파크,5400000,프레임무사고 확인,16,107792,999,경차,오토,가솔린
...,...,...,...,...,...,...,...,...,...,...
477,벤츠,E-클래스 W213,46990000,프레임무사고 확인,21,31579,1950,중형차,오토,디젤
478,BMW,X1 (E84),8200000,프레임무사고 확인,11,99995,1995,SUV,오토,디젤
479,미니,쿠퍼,12300000,프레임무사고 확인,15,54594,1499,소형차,오토,가솔린
480,폭스바겐,골프 7세대,8790000,프레임무사고 확인,15,142141,1395,준중형차,오토,가솔린


---

변속기가 오토가 아닌 게 있는지 확인

In [8]:
df[df['transmission']!="오토"]

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype


모두 오토임을 확인했다. 변속기 오토/수동 여부도 예측 성능에 도움을 주지 않을 것으로 예상된다.

---

다음은 fueltype이 전기차인 데이터를 조회

In [9]:
df[df['fueltype']=="전기"]

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype
104,제네시스,GV60,51300000,프레임무사고 확인,23,4076,111,SUV,오토,전기
124,현대,아이오닉5,24790000,프레임무사고 확인,21,108055,111,준중형차,오토,전기
133,현대,아이오닉5,36500000,프레임무사고 확인,24,32272,111,준중형차,오토,전기
164,현대,아이오닉6,38100000,프레임무사고 확인,22,28658,111,중형차,오토,전기
372,테슬라,모델 Y,48500000,프레임무사고 확인,23,23796,173,SUV,오토,전기
413,벤츠,EQS V297,81000000,프레임무사고 확인,23,20652,272,대형차,오토,전기
447,테슬라,모델 3,39900000,프레임무사고 확인,21,51382,225,준중형차,오토,전기


전기차인데 배기량이 존재한다.

In [10]:
df.loc[df['fueltype']=="전기", 'cc'] = 0

df[df['fueltype']=="전기"]

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype
104,제네시스,GV60,51300000,프레임무사고 확인,23,4076,0,SUV,오토,전기
124,현대,아이오닉5,24790000,프레임무사고 확인,21,108055,0,준중형차,오토,전기
133,현대,아이오닉5,36500000,프레임무사고 확인,24,32272,0,준중형차,오토,전기
164,현대,아이오닉6,38100000,프레임무사고 확인,22,28658,0,중형차,오토,전기
372,테슬라,모델 Y,48500000,프레임무사고 확인,23,23796,0,SUV,오토,전기
413,벤츠,EQS V297,81000000,프레임무사고 확인,23,20652,0,대형차,오토,전기
447,테슬라,모델 3,39900000,프레임무사고 확인,21,51382,0,준중형차,오토,전기


배기량을 모두 0으로 수정했다.

---

연식이 2000년보다 오래된 차량을 조회

In [11]:
df[df['year']>80], df[df['year']>90], df[df['year']==00]

(Empty DataFrame
 Columns: [brand, name, price, accident, year, distance, cc, kind, transmission, fueltype]
 Index: [],
 Empty DataFrame
 Columns: [brand, name, price, accident, year, distance, cc, kind, transmission, fueltype]
 Index: [],
 Empty DataFrame
 Columns: [brand, name, price, accident, year, distance, cc, kind, transmission, fueltype]
 Index: [])

2000년 이후 차량들만 있음을 확인했다.

---

### 원-핫 인코딩을 위한 전처리

#### 국산/외제 브랜드 (이진 분류)

In [12]:
df['brand'].unique()

array(['현대', '기아', '제네시스', '쉐보레(GM대우)', '르노코리아(삼성)', 'KG모빌리티(쌍용)', '벤츠',
       '마세라티', '포드', 'BMW', '캐딜락', '랜드로버', '지프', '폭스바겐', '아우디', '재규어',
       '볼보', '벤틀리', '미니', '테슬라', '포르쉐', '푸조', '동풍소콘', '링컨', '혼다',
       '시트로엥/DS', '인피니티', '애스턴마틴', '렉서스'], dtype=object)

분류가 어려운 쉐보레 데이터를 조회

In [13]:
df[df['brand']=="쉐보레(GM대우)"]

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype
4,쉐보레(GM대우),더 넥스트 스파크,5400000,프레임무사고 확인,16,107792,999,경차,오토,가솔린
41,쉐보레(GM대우),더 뉴 트랙스,10500000,프레임무사고 확인,19,87845,1362,SUV,오토,가솔린
50,쉐보레(GM대우),더 뉴 트랙스,11800000,프레임무사고 확인,20,42656,1362,SUV,오토,가솔린
57,쉐보레(GM대우),캡티바,8200000,프레임무사고 확인,16,131700,1956,SUV,오토,디젤
71,쉐보레(GM대우),올 뉴 말리부,10300000,프레임무사고 확인,17,107454,1998,중형차,오토,가솔린
78,쉐보레(GM대우),트랙스 크로스오버,19200000,프레임무사고 확인,23,35334,1199,SUV,오토,가솔린
109,쉐보레(GM대우),더 뉴 트랙스,9500000,프레임무사고 확인,19,76642,1598,SUV,오토,디젤
120,쉐보레(GM대우),트레일블레이저,21300000,프레임무사고 확인,21,26498,1341,SUV,오토,가솔린
150,쉐보레(GM대우),올 뉴 카마로,26990000,프레임무사고 확인,17,56410,6162,스포츠카,오토,가솔린
151,쉐보레(GM대우),더 뉴 트랙스,11390000,프레임무사고 확인,18,37868,1362,SUV,오토,가솔린


카마로를 제외한 나머지 모델은 한국 공장에서 생산하는 한국 모델이다.
따라서 카마로를 미국GM(Chevrolet), 나머지를 한국GM(GM대우)으로 재분류

In [14]:
df.loc[df['name']=="올 뉴 카마로", "brand"] = "쉐보레"
df.loc[df['brand']=="쉐보레(GM대우)", "brand"] = "GM대우"

df[df['brand']=="GM대우"]

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype
4,GM대우,더 넥스트 스파크,5400000,프레임무사고 확인,16,107792,999,경차,오토,가솔린
41,GM대우,더 뉴 트랙스,10500000,프레임무사고 확인,19,87845,1362,SUV,오토,가솔린
50,GM대우,더 뉴 트랙스,11800000,프레임무사고 확인,20,42656,1362,SUV,오토,가솔린
57,GM대우,캡티바,8200000,프레임무사고 확인,16,131700,1956,SUV,오토,디젤
71,GM대우,올 뉴 말리부,10300000,프레임무사고 확인,17,107454,1998,중형차,오토,가솔린
78,GM대우,트랙스 크로스오버,19200000,프레임무사고 확인,23,35334,1199,SUV,오토,가솔린
109,GM대우,더 뉴 트랙스,9500000,프레임무사고 확인,19,76642,1598,SUV,오토,디젤
120,GM대우,트레일블레이저,21300000,프레임무사고 확인,21,26498,1341,SUV,오토,가솔린
151,GM대우,더 뉴 트랙스,11390000,프레임무사고 확인,18,37868,1362,SUV,오토,가솔린
157,GM대우,더 뉴 스파크,7700000,프레임무사고 확인,19,59015,999,경차,오토,가솔린


In [15]:
df[df['brand']=="쉐보레"]

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype
150,쉐보레,올 뉴 카마로,26990000,프레임무사고 확인,17,56410,6162,스포츠카,오토,가솔린


둘을 국산과 외제로 이진분류하였다.

---

#### 차종

In [16]:
df['kind'].unique()

array(['대형차', '중형차', '경차', 'SUV', 'RV', '준중형차', '스포츠카', '소형차', '화물차'],
      dtype=object)

대형 화물차는 조사 대상이 아니므로 정보 조회를 해보았다.

In [17]:
df[df['kind']=="화물차"]

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype
430,포드,레인저 4세대,63500000,프레임무사고 확인,23,15849,1996,화물차,오토,디젤
445,지프,글래디에이터 (JT),44700000,프레임무사고 확인,21,72978,3604,화물차,오토,가솔린


검색 결과, 포드 레인저와 지프 글레디에이터는 대형 화물차가 아닌 픽업트럭이었다.

---

#### 연료

In [18]:
df['fueltype'].unique()

array(['가솔린', '디젤', '가솔린+전기', 'LPG(일반인 ', '전기'], dtype=object)

가솔린, 디젤, 전기뿐만 아니라 하이브리드와 LPG도 확인.   
가솔린+전기는 '하이브리드'로 수정

In [19]:
df[df['fueltype']=="가솔린+전기"].head()

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype
8,현대,투싼 하이브리드 (NX4),24900000,프레임무사고 확인,20,102792,1598,SUV,오토,가솔린+전기
11,기아,디 올 뉴 니로,32900000,프레임무사고 확인,22,20181,1580,SUV,오토,가솔린+전기
23,기아,K7 프리미어 하이브리드,26700000,프레임무사고 확인,20,70296,2359,대형차,오토,가솔린+전기
25,기아,K8 하이브리드,37300000,프레임무사고 확인,22,33022,1598,대형차,오토,가솔린+전기
27,기아,올 뉴 K7 하이브리드,14590000,프레임무사고 확인,17,121698,2359,대형차,오토,가솔린+전기


In [20]:
df.loc[df['fueltype']=="가솔린+전기", "fueltype"] = "하이브리드"

df[df['fueltype']=="하이브리드"].head()

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype
8,현대,투싼 하이브리드 (NX4),24900000,프레임무사고 확인,20,102792,1598,SUV,오토,하이브리드
11,기아,디 올 뉴 니로,32900000,프레임무사고 확인,22,20181,1580,SUV,오토,하이브리드
23,기아,K7 프리미어 하이브리드,26700000,프레임무사고 확인,20,70296,2359,대형차,오토,하이브리드
25,기아,K8 하이브리드,37300000,프레임무사고 확인,22,33022,1598,대형차,오토,하이브리드
27,기아,올 뉴 K7 하이브리드,14590000,프레임무사고 확인,17,121698,2359,대형차,오토,하이브리드


다음은 LPG 조회

In [21]:
df[df['fueltype']=="LPG(일반인 "]

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype
34,현대,더 뉴 아반떼 AD,9200000,프레임무사고 확인,19,97674,1591,준중형차,오토,LPG(일반인
39,기아,K5 3세대,15200000,프레임무사고 확인,20,61367,1999,중형차,오토,LPG(일반인
89,현대,그랜저 IG,20600000,프레임무사고 확인,18,33933,2999,대형차,오토,LPG(일반인
92,르노코리아(삼성),더 뉴 QM6,20000000,프레임무사고 확인,21,73056,1998,SUV,오토,LPG(일반인
95,현대,아반떼 (CN7),16500000,프레임무사고 확인,21,58165,1591,준중형차,오토,LPG(일반인
142,르노코리아(삼성),더 뉴 QM6,23850000,프레임무사고 확인,22,22151,1998,SUV,오토,LPG(일반인
167,르노코리아(삼성),더 뉴 QM6,19200000,프레임무사고 확인,21,42118,1998,SUV,오토,LPG(일반인
194,기아,K7 프리미어,17690000,프레임무사고 확인,20,96681,2999,대형차,오토,LPG(일반인
199,현대,쏘나타 뉴 라이즈,10500000,프레임무사고 확인,17,128263,1999,중형차,오토,LPG(일반인


'LPG(일반인 ' -> 'LPG'로 수정

In [22]:
df.loc[df['fueltype']=="LPG(일반인 ", "fueltype"] = "LPG"

df[df['fueltype']=="LPG"]

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype
34,현대,더 뉴 아반떼 AD,9200000,프레임무사고 확인,19,97674,1591,준중형차,오토,LPG
39,기아,K5 3세대,15200000,프레임무사고 확인,20,61367,1999,중형차,오토,LPG
89,현대,그랜저 IG,20600000,프레임무사고 확인,18,33933,2999,대형차,오토,LPG
92,르노코리아(삼성),더 뉴 QM6,20000000,프레임무사고 확인,21,73056,1998,SUV,오토,LPG
95,현대,아반떼 (CN7),16500000,프레임무사고 확인,21,58165,1591,준중형차,오토,LPG
142,르노코리아(삼성),더 뉴 QM6,23850000,프레임무사고 확인,22,22151,1998,SUV,오토,LPG
167,르노코리아(삼성),더 뉴 QM6,19200000,프레임무사고 확인,21,42118,1998,SUV,오토,LPG
194,기아,K7 프리미어,17690000,프레임무사고 확인,20,96681,2999,대형차,오토,LPG
199,현대,쏘나타 뉴 라이즈,10500000,프레임무사고 확인,17,128263,1999,중형차,오토,LPG


---

그럼 이제 본격적으로 원-핫 인코딩을 시작.

### X   
##### - brand: 국산(0), 외제(1)    
##### - name: 제거
##### - accident: 제거   
##### - year: 보존   
##### - distance: 보존   
##### - cc: 보존   
##### - kind:   
대형차(1 0 0 0 0 0 0 0 0)   
중형차(0 1 0 0 0 0 0 0 0)   
경차(0 0 1 0 0 0 0 0 0)   
SUV(0 0 0 1 0 0 0 0 0)   
RV  (0 0 0 0 1 0 0 0 0)   
준중형(0 0 0 0 0 1 0 0 0)   
스포츠카(0 0 0 0 0 0 1 0 0)   
소형차(0 0 0 0 0 0 0 1 0)   
화물차(0 0 0 0 0 0 0 0 1)   

##### - transmission: 제거   
##### - fueltype:   
가솔린(1 0 0 0 0)   
디젤(0 1 0 0 0)   
하이브리드(0 0 1 0 0)   
LPG(0 0 0 1 0)   
전기(0 0 0 0 1)   


   


### Y   
##### - price: 보존

---

#### brand 이진 분류

In [23]:
df['brand'].unique()

array(['현대', '기아', '제네시스', 'GM대우', '르노코리아(삼성)', 'KG모빌리티(쌍용)', '쉐보레', '벤츠',
       '마세라티', '포드', 'BMW', '캐딜락', '랜드로버', '지프', '폭스바겐', '아우디', '재규어',
       '볼보', '벤틀리', '미니', '테슬라', '포르쉐', '푸조', '동풍소콘', '링컨', '혼다',
       '시트로엥/DS', '인피니티', '애스턴마틴', '렉서스'], dtype=object)

In [24]:
df.loc[(df['brand']=="현대")|(df['brand']=="기아")|(df['brand']=="제네시스")|(df['brand']=="GM대우")|(df['brand']=="르노코리아(삼성)")|(df['brand']=="KG모빌리티(쌍용)"), 'brand'] = 0
df.loc[df['brand']!=0, 'brand']=1

df['brand'].unique()

array([0, 1], dtype=object)

변환 후 국산차 확인

In [25]:
df[df['brand']==0]

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype
0,0,더 뉴 그랜저 IG,26900000,프레임무사고 확인,20,26143,2497,대형차,오토,가솔린
1,0,K5 3세대,27900000,프레임무사고 확인,22,20222,1598,중형차,오토,가솔린
2,0,더 뉴 레이,12600000,프레임무사고 확인,20,15939,998,경차,오토,가솔린
3,0,G80 (RG3),37500000,프레임무사고 확인,21,84011,2497,대형차,오토,가솔린
4,0,더 넥스트 스파크,5400000,프레임무사고 확인,16,107792,999,경차,오토,가솔린
...,...,...,...,...,...,...,...,...,...,...
211,0,팰리세이드,35500000,프레임무사고 확인,22,59481,2199,SUV,오토,디젤
212,0,더 뉴 QM6,16900000,프레임무사고 확인,19,44965,1997,SUV,오토,가솔린
213,0,더 뉴 레이,13000000,프레임무사고 확인,21,51243,998,경차,오토,가솔린
214,0,쏘렌토 4세대,31300000,프레임무사고 확인,20,104754,2151,SUV,오토,디젤


외제차 확인

In [26]:
df[df['brand']==1]

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype
150,1,올 뉴 카마로,26990000,프레임무사고 확인,17,56410,6162,스포츠카,오토,가솔린
216,1,A-클래스 W176,17790000,프레임무사고 확인,17,11273,1595,소형차,오토,가솔린
217,1,S-클래스 W223,149000000,프레임무사고 확인,23,35294,2999,대형차,오토,가솔린
218,1,S-클래스 W223,142990000,프레임무사고 확인,23,17992,2999,대형차,오토,가솔린
219,1,E-클래스 W213,29900000,프레임무사고 확인,18,63133,1950,중형차,오토,디젤
...,...,...,...,...,...,...,...,...,...,...
477,1,E-클래스 W213,46990000,프레임무사고 확인,21,31579,1950,중형차,오토,디젤
478,1,X1 (E84),8200000,프레임무사고 확인,11,99995,1995,SUV,오토,디젤
479,1,쿠퍼,12300000,프레임무사고 확인,15,54594,1499,소형차,오토,가솔린
480,1,골프 7세대,8790000,프레임무사고 확인,15,142141,1395,준중형차,오토,가솔린


#### name, accident, transmission column 제거

In [27]:
df.drop(columns=['name', 'accident', 'transmission'], inplace=True)

df.head()

,brand,price,year,distance,cc,kind,fueltype
0,0,26900000,20,26143,2497,대형차,가솔린
1,0,27900000,22,20222,1598,중형차,가솔린
2,0,12600000,20,15939,998,경차,가솔린
3,0,37500000,21,84011,2497,대형차,가솔린
4,0,5400000,16,107792,999,경차,가솔린


#### kind, fueltype column 원-핫 인코딩

https://jimmy-ai.tistory.com/263

In [28]:
df = pd.get_dummies(df, columns=['kind', 'fueltype'])

df.head()

,brand,price,year,distance,cc,kind_RV,kind_SUV,kind_경차,kind_대형차,kind_소형차,kind_스포츠카,kind_준중형차,kind_중형차,kind_화물차,fueltype_LPG,fueltype_가솔린,fueltype_디젤,fueltype_전기,fueltype_하이브리드
0,0,26900000,20,26143,2497,False,False,False,True,False,False,False,False,False,False,True,False,False,False
1,0,27900000,22,20222,1598,False,False,False,False,False,False,False,True,False,False,True,False,False,False
2,0,12600000,20,15939,998,False,False,True,False,False,False,False,False,False,False,True,False,False,False
3,0,37500000,21,84011,2497,False,False,False,True,False,False,False,False,False,False,True,False,False,False
4,0,5400000,16,107792,999,False,False,True,False,False,False,False,False,False,False,True,False,False,False


---

### Scaler 표준화

In [29]:
from sklearn.preprocessing import StandardScaler

In [30]:
ss = StandardScaler()

df[['year', 'distance', 'cc']] = ss.fit_transform(df[['year', 'distance', 'cc']])

df

,brand,price,year,distance,cc,kind_RV,kind_SUV,kind_경차,kind_대형차,kind_소형차,kind_스포츠카,kind_준중형차,kind_중형차,kind_화물차,fueltype_LPG,fueltype_가솔린,fueltype_디젤,fueltype_전기,fueltype_하이브리드
0,0,26900000,0.132698,-1.031228,0.308276,False,False,False,True,False,False,False,False,False,False,True,False,False,False
1,0,27900000,0.917347,-1.195621,-0.761599,False,False,False,False,False,False,False,True,False,False,True,False,False,False
2,0,12600000,0.132698,-1.314536,-1.475642,False,False,True,False,False,False,False,False,False,False,True,False,False,False
3,0,37500000,0.525023,0.575441,0.308276,False,False,False,True,False,False,False,False,False,False,True,False,False,False
4,0,5400000,-1.436600,1.235706,-1.474452,False,False,True,False,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,1,46990000,0.525023,-0.880301,-0.342693,False,False,False,False,False,False,False,True,False,False,False,True,False,False
478,1,8200000,-3.398223,1.019227,-0.289140,False,True,False,False,False,False,False,False,False,False,False,True,False,False
479,1,12300000,-1.828925,-0.241304,-0.879416,False,False,False,False,True,False,False,False,False,False,True,False,False,False
480,1,8790000,-1.828925,2.189385,-1.003183,False,False,False,False,False,False,True,False,False,False,True,False,False,False


### Label(Y)인 price 칼럼을 분리

In [31]:
df_target = pd.DataFrame(df['price'])

df_target

,price
0,26900000
1,27900000
2,12600000
3,37500000
4,5400000
...,...
477,46990000
478,8200000
479,12300000
480,8790000


df에서 price 칼럼 제거

In [32]:
df.drop(columns=['price'], inplace=True)

df

,brand,year,distance,cc,kind_RV,kind_SUV,kind_경차,kind_대형차,kind_소형차,kind_스포츠카,kind_준중형차,kind_중형차,kind_화물차,fueltype_LPG,fueltype_가솔린,fueltype_디젤,fueltype_전기,fueltype_하이브리드
0,0,0.132698,-1.031228,0.308276,False,False,False,True,False,False,False,False,False,False,True,False,False,False
1,0,0.917347,-1.195621,-0.761599,False,False,False,False,False,False,False,True,False,False,True,False,False,False
2,0,0.132698,-1.314536,-1.475642,False,False,True,False,False,False,False,False,False,False,True,False,False,False
3,0,0.525023,0.575441,0.308276,False,False,False,True,False,False,False,False,False,False,True,False,False,False
4,0,-1.436600,1.235706,-1.474452,False,False,True,False,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,1,0.525023,-0.880301,-0.342693,False,False,False,False,False,False,False,True,False,False,False,True,False,False
478,1,-3.398223,1.019227,-0.289140,False,True,False,False,False,False,False,False,False,False,False,True,False,False
479,1,-1.828925,-0.241304,-0.879416,False,False,False,False,True,False,False,False,False,False,True,False,False,False
480,1,-1.828925,2.189385,-1.003183,False,False,False,False,False,False,True,False,False,False,True,False,False,False


---

국산/외제 분포 확인

In [33]:
import plotly.express as px

fig = px.histogram(df, x="brand")
fig.show()

In [34]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = df
y = df_target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [35]:
X_test

,brand,year,distance,cc,kind_RV,kind_SUV,kind_경차,kind_대형차,kind_소형차,kind_스포츠카,kind_준중형차,kind_중형차,kind_화물차,fueltype_LPG,fueltype_가솔린,fueltype_디젤,fueltype_전기,fueltype_하이브리드
168,0,0.525023,-0.516948,0.308276,False,True,False,False,False,False,False,False,False,False,True,False,False,False
101,0,0.917347,0.049167,1.466216,False,False,False,True,False,False,False,False,False,False,True,False,False,False
308,1,-1.044276,0.702380,-0.113009,False,False,False,False,False,False,True,False,False,False,False,True,False,False
322,1,-1.436600,-0.332426,0.902122,False,False,False,False,False,False,False,True,False,False,True,False,False,False
100,0,0.132698,-0.101427,0.308276,False,False,False,True,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,1,0.917347,-0.357026,-0.289140,False,False,False,True,False,False,False,False,False,False,False,True,False,False
117,0,-1.436600,0.776094,-0.658062,False,True,False,False,False,False,False,False,False,False,False,True,False,False
26,0,0.917347,0.917082,0.308276,False,True,False,False,False,False,False,False,False,False,True,False,False,False
152,0,-1.044276,0.762573,-0.285570,False,False,False,False,False,False,False,True,False,False,True,False,False,False


In [36]:
y_test

,price
168,41000000
101,80990000
308,20300000
322,32500000
100,42800000
...,...
471,52500000
117,11900000
26,32000000
152,19800000


In [37]:
y_pred

array([[35216383.6945212 ],
       [60116482.79582793],
       [26981169.83662785],
       [38946306.42828386],
       [37808822.46617446],
       [19134766.86676757],
       [18837768.42487156],
       [40554850.40713571],
       [69441706.99507181],
       [20296668.91029259],
       [93674351.86059311],
       [40299443.93775705],
       [19851711.27896228],
       [58126390.73159708],
       [57407606.63828391],
       [-4571140.6821599 ],
       [42733718.85356601],
       [91513502.67398751],
       [38586780.33290069],
       [21727774.7466104 ],
       [14133676.74356726],
       [37154065.5249206 ],
       [81013426.10667688],
       [21941056.48503058],
       [42634001.687761  ],
       [ 6600522.8912352 ],
       [22010017.76317468],
       [31940244.19692098],
       [ 5226080.75436747],
       [35673791.72536045],
       [22421674.06541232],
       [45201289.37222342],
       [43766496.89511972],
       [-7342833.8576384 ],
       [69083401.91662432],
       [46893562.831

가격이 음수일 수는 없다. 모델 학습에 문제가 있음을 확인했다.

In [38]:
df_target[df_target["price"]<0]

,price


In [39]:
from sklearn.metrics import mean_absolute_error

In [40]:
mse = mean_squared_error(y_test, y_pred)

print(f"{mse:.2f}")

253903660150093.78


y값이 백만~천만 단위로 매우 크기 때문에 평균제곱오차가 매우 크다.

In [41]:
mae = mean_absolute_error(y_test, y_pred)

print(f"{mae:.2f}")

10837278.26


평균절댓값오차는 1,000만원을 넘긴다.

---

### 랜덤포레스트 회귀모델로 더 나은 성능의 모델을 구축

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

y = y.values

y = y.ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


""" n_estimators = 100 으로 임의 실행"""
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

rf_regressor.fit(X_train, y_train)

y_pred = rf_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae:.4f}")


Mean Absolute Error: 7729765.0350


##### 하이퍼파라미터 튜닝(n_estimators)

In [49]:
from sklearn.model_selection import GridSearchCV

params = {'n_estimators': [i for i in range(0, 200)]}

gridsearch = GridSearchCV(estimator=rf_regressor, param_grid=params, cv=10)
gridsearch.fit(X_train, y_train)


/home/ms/venv/ml_venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning:


10 fits failed out of a total of 2000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/home/ms/venv/ml_venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/ms/venv/ml_venv/lib/python3.10/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/home/ms/venv/ml_venv/lib/python3.10/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/home/ms

GridSearchCV(cv=10, estimator=RandomForestRegressor(random_state=42),
             param_grid={'n_estimators': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                          12, 13, 14, 15, 16, 17, 18, 19, 20,
                                          21, 22, 23, 24, 25, 26, 27, 28, 29, ...]})

In [50]:
gridsearch.best_estimator_

RandomForestRegressor(n_estimators=23, random_state=42)

In [51]:
gridsearch.best_score_

0.7416273397817769

In [52]:
gridsearch.best_params_

{'n_estimators': 23}

In [53]:
best_n_estimators = gridsearch.best_params_["n_estimators"]
rf_regressor_optimized = RandomForestRegressor(n_estimators=best_n_estimators, random_state=42)

rf_regressor_optimized.fit(X_train, y_train)

y_pred_optimized = rf_regressor_optimized.predict(X_test)

optimized_mae = mean_absolute_error(y_test, y_pred_optimized)

print(f"optimized_mae: {optimized_mae:.2f}")

optimized_mae: 7733046.52
